# 1️⃣ Training an Adapter for a Transformer model

In this notebook, we train an adapter for a **RoBERTa** ([Liu et al., 2019](https://arxiv.org/pdf/1907.11692.pdf)) model for sequence classification on a **sentiment analysis** task using [adapter-transformers](https://github.com/Adapter-Hub/adapter-transformers), the _AdapterHub_ adaptation of HuggingFace's _transformers_ library.

If you're unfamiliar with the theoretical parts of adapters or the AdapterHub framework, check out our [introductory blog post](https://adapterhub.ml/blog/2020/11/adapting-transformers-with-adapterhub/) first.

We train a **Task Adapter** for a pre-trained model here. Most of the code is identical to a full finetuning setup using HuggingFace's transformers. For comparison, have a look at the [same guide using full finetuning](https://colab.research.google.com/drive/1brXJg5Mokm8h3shxqPRnoIsRwHQoncus?usp=sharing).

For training, we use the [movie review dataset by Pang and Lee (2005)](http://www.cs.cornell.edu/people/pabo/movie-review-data/). It contains movie reviews  from Rotten Tomatoes which are either classified as positive or negative. We download the dataset via HuggingFace's [datasets](https://github.com/huggingface/datasets) library.

## Installation

First, let's install the required libraries:

In [1]:
!pip install -U adapter-transformers
!pip install datasets

## Dataset Preprocessing

Before we start to train our adapter, we first prepare the training data. Our training dataset can be loaded via HuggingFace `datasets` using one line of code:

In [2]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("vannacute/AmazonReviewHelpfulness")
dataset.num_rows

Using custom data configuration vannacute___AmazonReviewHelpfulness-303f37a529acb78b
Reusing dataset json (C:\Users\Junfe\.cache\huggingface\datasets\json\vannacute___AmazonReviewHelpfulness-303f37a529acb78b\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)


  0%|          | 0/3 [00:00<?, ?it/s]

{'train': 115251, 'test': 25000, 'validation': 5000}

Every dataset sample has an input text and a binary label:

In [3]:
# dataset['train']['label']
l = dataset['validation'].map(lambda x : torch.tensor([False]) if x=='unhelpful' else x) 

Loading cached processed dataset at C:\Users\Junfe\.cache\huggingface\datasets\json\vannacute___AmazonReviewHelpfulness-303f37a529acb78b\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-358380962e9a411c.arrow


In [4]:
def maplabel(example):
  if example['label'] == "helpful":
    example['label'] = True
  else:
    example['label'] = False
  return example

dataset['train'] = dataset['train'] .map(maplabel)

Loading cached processed dataset at C:\Users\Junfe\.cache\huggingface\datasets\json\vannacute___AmazonReviewHelpfulness-303f37a529acb78b\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-cafbfb4aaccebbe1.arrow


In [5]:
dataset['validation'] = dataset['validation'] .map(maplabel)
dataset['test'] = dataset['test'] .map(maplabel)

Loading cached processed dataset at C:\Users\Junfe\.cache\huggingface\datasets\json\vannacute___AmazonReviewHelpfulness-303f37a529acb78b\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-c9cca26846ef6e7c.arrow
Loading cached processed dataset at C:\Users\Junfe\.cache\huggingface\datasets\json\vannacute___AmazonReviewHelpfulness-303f37a529acb78b\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-de634f1ae6dbf563.arrow


In [6]:
from datasets import ClassLabel, Value
for name in ['train','validation','test']:
  new_features = dataset[name].features.copy()
  new_features["label"] = ClassLabel(names=['neg', 'pos'])
  dataset[name] = dataset[name].cast(new_features)
  dataset[name].features

Loading cached processed dataset at C:\Users\Junfe\.cache\huggingface\datasets\json\vannacute___AmazonReviewHelpfulness-303f37a529acb78b\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-9aa234147fb8c46e.arrow
Loading cached processed dataset at C:\Users\Junfe\.cache\huggingface\datasets\json\vannacute___AmazonReviewHelpfulness-303f37a529acb78b\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-cd2b31a31ac7f2b3.arrow
Loading cached processed dataset at C:\Users\Junfe\.cache\huggingface\datasets\json\vannacute___AmazonReviewHelpfulness-303f37a529acb78b\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-3c5f6ab84b09a825.arrow


Now, we need to encode all dataset samples to valid inputs for our Transformer model. Since we want to train on `roberta-base`, we load the corresponding `RobertaTokenizer`. Using `dataset.map()`, we can pass the full dataset through the tokenizer in batches:

In [7]:
import numpy as np
nLabels = len(np.unique(dataset['train']['label']))
nLabels

2

In [8]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=512, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Loading cached processed dataset at C:\Users\Junfe\.cache\huggingface\datasets\json\vannacute___AmazonReviewHelpfulness-303f37a529acb78b\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-fd26f8d8458f9cc3.arrow
Loading cached processed dataset at C:\Users\Junfe\.cache\huggingface\datasets\json\vannacute___AmazonReviewHelpfulness-303f37a529acb78b\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-63cd9525b0016482.arrow


  0%|          | 0/5 [00:00<?, ?ba/s]

C:\Users\Junfe\AppData\Local\Temp/ipykernel_40320/1657904570.py:12: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use DatasetDict.rename_column instead.
  dataset.rename_column_("label", "labels")


Now we're ready to train our model...

## Training

We use a pre-trained RoBERTa model from HuggingFace. We use `RobertaModelWithHeads`, a class unique to `adapter-transformers`, which allows us to add and configure prediction heads in a flexibler way.

In [9]:
# dataset['train']['labels']

In [10]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=2,
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

**Here comes the important part!**

We add a new adapter to our model by calling `add_adapter()`. We pass a name (`"rotten_tomatoes"`) and [the type of adapter](https://docs.adapterhub.ml/adapters.html#adapter-types) (task adapter). Next, we add a binary classification head. It's convenient to give the prediction head the same name as the adapter. This allows us to activate both together in the next step. The `train_adapter()` method does two things:

1. It freezes all weights of the pre-trained model so only the adapter weights are updated during training.
2. It activates the adapter and the prediction head such that both are used in every forward pass.

In [11]:
adaptername = 'Helpfulness'

# Add a new adapter
model.add_adapter(adaptername)
# Add a matching classification head
model.add_classification_head(
    adaptername,
    num_labels=2
    # id2label={ 0: "👎", 1: "👍"}
  )
# Activate the adapter
model.train_adapter(adaptername)

For training, we make use of the `Trainer` class built-in into `transformers`. We configure the training process using a `TrainingArguments` object and define a method that will calculate the evaluation accuracy in the end. We pass both, together with the training and validation split of our dataset, to the trainer instance.

**Note the differences in hyperparameters compared to full finetuning.** Adapter training usually required a few more training epochs than full finetuning.

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'labels', 'text'],
        num_rows: 115251
    })
    test: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'labels', 'text'],
        num_rows: 25000
    })
    validation: Dataset({
        features: ['attention_mask', 'id', 'input_ids', 'labels', 'text'],
        num_rows: 5000
    })
})

In [13]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

Start the training 🚀

In [14]:
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

In [15]:
trainer.train()

***** Running training *****
  Num examples = 115251
  Num Epochs = 6
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 57630


Step,Training Loss
200,0.440500
400,0.353100
600,0.356100
800,0.371100
1000,0.357200
1200,0.332900
1400,0.338200
1600,0.356600
1800,0.341200
2000,0.325100


Saving model checkpoint to ./training_output\checkpoint-500
Configuration saved in ./training_output\checkpoint-500\Helpfulness\adapter_config.json
Module weights saved in ./training_output\checkpoint-500\Helpfulness\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-500\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-500\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-500\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-500\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-500\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-500\Helpfulness\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-1000
Configuration saved in ./training_output\checkpoint-1000\Helpfulness\adapter_config.json
Module weights saved in ./training_output\checkpoint-1000\Helpfulness\pytorch_adap

Module weights saved in ./training_output\checkpoint-5500\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-5500\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-5500\Helpfulness\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-6000
Configuration saved in ./training_output\checkpoint-6000\Helpfulness\adapter_config.json
Module weights saved in ./training_output\checkpoint-6000\Helpfulness\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-6000\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-6000\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-6000\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-6000\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-6000\Helpfulness\head_config.json
Module weights saved in ./training_output\ch

Module weights saved in ./training_output\checkpoint-11000\Helpfulness\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-11000\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-11000\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-11000\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-11000\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-11000\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-11000\Helpfulness\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-11500
Configuration saved in ./training_output\checkpoint-11500\Helpfulness\adapter_config.json
Module weights saved in ./training_output\checkpoint-11500\Helpfulness\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-11500\Helpfulness\head_config.json
Module weights saved in ./training_o

...

Configuration saved in ./training_output\checkpoint-37000\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-37000\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-37000\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-37000\Helpfulness\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-37500
Configuration saved in ./training_output\checkpoint-37500\Helpfulness\adapter_config.json
Module weights saved in ./training_output\checkpoint-37500\Helpfulness\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-37500\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-37500\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-37500\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-37500\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training

Saving model checkpoint to ./training_output\checkpoint-42500
Configuration saved in ./training_output\checkpoint-42500\Helpfulness\adapter_config.json
Module weights saved in ./training_output\checkpoint-42500\Helpfulness\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-42500\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-42500\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-42500\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-42500\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-42500\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-42500\Helpfulness\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-43000
Configuration saved in ./training_output\checkpoint-43000\Helpfulness\adapter_config.json
Module weights saved in ./training_output\checkpoint-43000\Hel

Configuration saved in ./training_output\checkpoint-47500\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-47500\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-47500\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-47500\Helpfulness\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-48000
Configuration saved in ./training_output\checkpoint-48000\Helpfulness\adapter_config.json
Module weights saved in ./training_output\checkpoint-48000\Helpfulness\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-48000\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-48000\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-48000\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-48000\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training

Saving model checkpoint to ./training_output\checkpoint-53000
Configuration saved in ./training_output\checkpoint-53000\Helpfulness\adapter_config.json
Module weights saved in ./training_output\checkpoint-53000\Helpfulness\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-53000\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-53000\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-53000\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-53000\Helpfulness\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-53000\Helpfulness\head_config.json
Module weights saved in ./training_output\checkpoint-53000\Helpfulness\pytorch_model_head.bin
Saving model checkpoint to ./training_output\checkpoint-53500
Configuration saved in ./training_output\checkpoint-53500\Helpfulness\adapter_config.json
Module weights saved in ./training_output\checkpoint-53500\Hel

TrainOutput(global_step=57630, training_loss=0.2946141730988168, metrics={'train_runtime': 11727.7217, 'train_samples_per_second': 58.963, 'train_steps_per_second': 4.914, 'total_flos': 1.85097723416064e+17, 'train_loss': 0.2946141730988168, 'epoch': 6.0})

Looks good! Let's evaluate our adapter on the validation split of the dataset to see how well it learned:

In [16]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 5000
  Batch size = 12


{'eval_loss': 0.319141685962677,
 'eval_acc': 0.8724,
 'eval_runtime': 37.6691,
 'eval_samples_per_second': 132.735,
 'eval_steps_per_second': 11.07,
 'epoch': 6.0}

We can put our trained model into a `transformers` pipeline to be able to make new predictions conveniently:

In [17]:
# from transformers import TextClassificationPipeline

# classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

# classifier("This is awesome!")

At last, we can also extract the adapter from our model and separately save it for later reuse. Note the size difference compared to a full model!

In [18]:
model.save_adapter("./final_adapter", adaptername)
!ls -lh final_adapter

Configuration saved in ./final_adapter\adapter_config.json
Module weights saved in ./final_adapter\pytorch_adapter.bin
Configuration saved in ./final_adapter\head_config.json
Module weights saved in ./final_adapter\pytorch_model_head.bin
'ls' is not recognized as an internal or external command,
operable program or batch file.


In [19]:
(pred, label, metric) = trainer.predict(dataset["test"])

***** Running Prediction *****
  Num examples = 25000
  Batch size = 12


In [23]:
pred_label = np.argmax(pred, axis=1)

In [25]:
from sklearn.metrics import classification_report

print(classification_report(label, pred_label, digits = 4))

              precision    recall  f1-score   support

           0     0.6538    0.3416    0.4487      3665
           1     0.8955    0.9689    0.9308     21335

    accuracy                         0.8770     25000
   macro avg     0.7746    0.6553    0.6897     25000
weighted avg     0.8600    0.8770    0.8601     25000



**Share your work!**

The next step after training is to share our adapter with the world via _AdapterHub_. [Read our guide](https://docs.adapterhub.ml/contributing.html) on how to prepare the adapter module we just saved and contribute it to the Hub!

➡️ Also continue with [the next Colab notebook](https://colab.research.google.com/github/Adapter-Hub/adapter-transformers/blob/master/notebooks/02_Adapter_Inference.ipynb) to learn how to use adapters from the Hub.